# Keys to Python
-  Python doesn't use compiler so it doesn't have programmable access to hardware optimization
    - Compiler's access to the details of the underlying platform which means it can utilize optimizations that exploit chip-specific features and cache memory. 
-  it provides the software **glue** that permits easy exchange of methods and data across core routines typically written in Fortran or C.
- Python is an interpreted language. This means that Python codes run on a Python virtual machine that provides a layer of abstraction between the code and the platform it runs on, thus making codes portable across different platforms.

# Numpy 
- Numpy is the de facto standard for numerical arrays in Python. 

In [1]:
import numpy as np 

x = np.array([1, 2, 4], dtype = np.float32)
x.itemsize # 4 bytes per float32 number

4

## ufuncs (Comprehensive Set of Universal Functions)

### Arithmetic Operations
- Addition `np.add(a, b)`
- Subtraction `np.subtract(a, b)`
- Multiplication `np.multiply(a, b)` (element-wise)
- Division `np.divide(a, b)`
### Trigonometric Functions
- Sine `np.sin(x)`
- Cosine `np.cos(x)`
- Tangent `np.tan(x)`
### Hyperbolic Functions
- sinh `np.sinh(x)`
- cosh `np.cosh(x)`
### Exponential and Logarithmic Functions
- Exponential `np.exp(x)`
- Natural Algorithm `np.log(x)`
- Base-10 Log `np.log10(x)`

### Aggregate Functions 
#### these perform operations over an entire array or specific axes
- Sum `np.sum(x)`
- Mean `np.mean(x)`
- Standard Deviation `np.std(x)`
- Minimum `np.min(x)` and Maximum `np.max(x)`

### Comparison Functions
- `np.greater([2, 3, 4], [1, 3, 2])  # Output: array([ True, False,  True])`
- `np.equal([2, 3, 4], [1, 3, 2])  # Output: array([False,  True, False])`

### Bitwise Functions
- Bitwise AND `np.bitwise_and(x, y)`
- Bitwise OR `np.bitwise_or(x, y)`
- Bitwise XOR `np.bitwise_xor(x, y)`
- Bitwise NOT `np.bitwise_not(x)`

### Absolute Value
- Absolute Value `np.abs(x)`

In [2]:
np.sin(x) # faster than math.sin, no explicit for loop

array([ 0.841471 ,  0.9092974, -0.7568025], dtype=float32)

### NP is limited to 32 dimensions unless we build it for more.

In [3]:
x = np.array([[1, 2, 3], [4, 5, 6]])
print(f"Shape: {x.shape} First Row: {x[0]} First Col: {x[:, 0]}")
print(f"All Rows but stride = 2: \n{x[:, ::2]}")
print(f"Reverse Rows \n{x[::-1]}")
print(f"Reverse Cols \n{x[:, ::-1]}")

Shape: (2, 3) First Row: [1 2 3] First Col: [1 4]
All Rows but stride = 2: 
[[1 3]
 [4 6]]
Reverse Rows 
[[4 5 6]
 [1 2 3]]
Reverse Cols 
[[3 2 1]
 [6 5 4]]


### Numpy uses pass-by-reference semantics
- Slice operation are **views** into the array without implicitly copying 

#### Cases Creating Copies
1. Using a list or a similar sequence type (but not a tuple) for indexing

In [13]:
arr = np.array([1, 2, 3, 4, 5])
indices = [0, 2, 4]  # non-tuple sequence
new_arr = arr[indices]
new_arr

array([1, 3, 5])

2. **Use another NumPy array**, which can be either of integer indices or boolean values, for indexing.

In [5]:
index_arr = np.array([0, 2, 4])
boolean_idx = np.array([True, False, True, False, True])

arr_by_int_np = arr[index_arr]
print(f"By int nparr:{arr_by_int_np}")
arr_by_bool = arr[boolean_idx]
print(f"By boolean nparr:{arr_by_bool}")

By int nparr:[1 3 5]
By boolean nparr:[1 3 5]


3. If you use a tuple for indexing, and **this tuple contains either a sequence (like a list) or a NumPy array**, it also results in a copy.

In [6]:
arr_2d = np.array([[1, 2], [3, 4], [5, 6]])
# Selecting the elements at (0, 1), (2, 1), (1, 1)
indices = ([0, 2, 1], [1, 1, 1])  # Tuple with two lists
new_arr = arr_2d[indices]
print(new_arr)

[2 6 4]


#### We construct y by slicing (which is a view), then any change will affect y
#### ```x.copy()``` explicitly force a copy 
#### ```y.flags``` sort this out

In [14]:
x = np.ones((3, 3))
y = x[:2, :2]
print(f"original y : \n{y}\nThen we make a change to x")
x[0, 0] = 999
print(f"y after change x: \n{y}\ny.flags: \n{y.flags}")

original y : 
[[1. 1.]
 [1. 1.]]
Then we make a change to x
y after change x: 
[[999.   1.]
 [  1.   1.]]
y.flags: 
  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False


In [8]:
y = x.copy()
y.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

### Overlapping Blocks (using view. NOT a copy)

Bear in mind that ```as_strided``` **DOESN't** check that you stay within memory block bounds. 
So, if the size of the target matrix is not filled by the available data, the **remaining elements will come from whatever bytes are at that memory location**.

In [9]:
from numpy.lib.stride_tricks import as_strided
x = np.arange(16, dtype = np.int64) # one int is 8-byte
print(f"x : {x}")

# (7, 4) stands for Shape
# (16, 8) stands for strides which means the resulting arr steps 8 bytes in the cols, and steps 16 bytes in the rows
y = as_strided(x, (7, 4), (16, 8))
y

x : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]


array([[ 0,  1,  2,  3],
       [ 2,  3,  4,  5],
       [ 4,  5,  6,  7],
       [ 6,  7,  8,  9],
       [ 8,  9, 10, 11],
       [10, 11, 12, 13],
       [12, 13, 14, 15]])


#### Multiplication Operation ```A@x``` or ```A.dot(x)```

In [10]:
A = np.array([[1, 2, 3, 4], [3, 4, 5, 6], [6, 7, 8, 9]])
x = np.array([1, 0, 0, 0])
A@x
A.dot(x)

array([1, 3, 6])

### Boardcasting

#### ```None``` tells Numpy to make copies of y along this dimension to create a conformable calculation.

In [11]:
x = np.array([0, 1])
y = np.array([0, 1, 2]).reshape([-1, 1])
print(f"x + y: \n{x + y}")

x = np.array([0, 1])
y = np.array([0, 1, 2])
print("Add one dimension to y and add one dim to x")
x + y[:, None], x[:, None] + y

x + y: 
[[0 1]
 [1 2]
 [2 3]]
Add one dimension to y and add one dim to x


(array([[0, 1],
        [1, 2],
        [2, 3]]),
 array([[0, 1, 2],
        [1, 2, 3]]))

In [12]:
z = np.array([0, 1, 2, 3])
x + y[:, None] + z[:, None, None]

array([[[0, 1],
        [1, 2],
        [2, 3]],

       [[1, 2],
        [2, 3],
        [3, 4]],

       [[2, 3],
        [3, 4],
        [4, 5]],

       [[3, 4],
        [4, 5],
        [5, 6]]])

# Numpy Masked Arrays



In [26]:
from numpy import ma

x = np.arange(10)
y = ma.masked_array(x, x < 5)  # y is also a view but not a copy.
y

masked_array(data=[--, --, --, --, --, 5, 6, 7, 8, 9],
             mask=[ True,  True,  True,  True,  True, False, False, False,
                   False, False],
       fill_value=999999)

In [27]:
y[-1] = 999
x

array([  0,   1,   2,   3,   4,   5,   6,   7,   8, 999])

# Floating - Point Numbers
## IEEE 754 Standard 
- **Sign bit**: This is the first bit and is used to represent the sign of the number. If the sign bit is 0, the number is positive; if the sign bit is 1, the number is negative.

- **Exponent bits**: The following bits represent the exponent. The number of bits used for the exponent depends on the precision of the floating-point representation (single precision, double precision, etc.). 
    - For single precision (32-bit), 8 bits are used for the exponent. For double precision (64-bit), 11 bits are used. 
    - The exponent is stored in "biased" form, meaning that **a bias is subtracted to get the actual exponent**. **For single precision, the bias is 127, and for double precision, it's 1023.**

- **Significand (Mantissa) bits**: The remaining bits are used for the significand. This is the fraction part of the number, **following a leading 1** (except for numbers very close to 0, known as subnormal numbers). In single precision, 23 bits are used for the significand, while in double precision, 52 bits are used.

---

To convert a decimal number to its binary IEEE 754 representation, follow these steps:

1. Determine the sign bit from the sign of the number.
2. Convert the absolute value of the number to binary form.
3. Normalize the binary number so that it's in the form of 1.xxxx... and find the binary exponent.
4. Adjust the exponent by the bias and convert it to binary.
5. Write down the sign bit, followed by the biased exponent bits, followed by the significand bits (without the leading 1).

---

Here's an example of converting the decimal number 0.375 to binary using IEEE 754 32-bit format:

1. The number is positive, so the sign bit is 0.
2. 0.375 in binary is 0.011 (because 0.375 = 1/4 + 1/8 = 2^-2 + 2^-3).
3. The normalized form is 1.1 x 2^-2, so the significand is 1.1.
4. For single precision, the exponent bias is 127. Our exponent is -2, so the biased exponent is 127 - 2 = 125, which is 01111101 in binary.
5. The significand (mantissa) without the leading 1 is .1 (in binary), which needs to be represented with 23 bits after the point, so it's 10000000000000000000000.

So the IEEE 754 32-bit binary representation of 0.375 is `0 01111101 10000000000000000000000`

In [28]:
.1+.2

0.30000000000000004

Let's express 0.1 as binary: $\approx 1.6 \times 2 ^{-4}$

In [43]:
a = 0.1
bits = []
while a > 0:
    q, a = divmod(a * 2, 1)
    bits.append(q)

print("".join(["%d"%i for i in bits]))

0001100110011001100110011001100110011001100110011001101


Also $0.2 \approx 1.6 \times 2^{-3}$ 

In [44]:
a = 0.2
bits = []
while a > 0:
    q, a = divmod(a * 2, 1)
    bits.append(q)

print("".join(["%d"%i for i in bits]))

001100110011001100110011001100110011001100110011001101


We only have 23 bits for the fractional parts so we need to **round off at 23 bits**.
Therefore, $0.1\approx (1.10011001100110011001101)_2 \times 2 ^{-4}$
This is the reason why decimal fraction calculation is not precise !

---

### Principle: Round to Nearest, Ties to Even
1. Round to Nearest:
- If the bit after the last representable bit (the one that we can't store, often called the "rounding bit") is 0, we do nothing; the value stays the same.
- If the rounding bit is 1 and any following bits are 1 (meaning the actual value is more than halfway to the next representable number), we round up by adding 1 to the last representable bit.
2. Ties to Even:

- If the rounding bit is 1 but all bits after it are 0 (it's exactly halfway between two representable numbers), we look at the last representable bit. If it's already even (0), we do nothing; if it's odd (1), we round up to make it even.


### For most practical purposes, these tiny errors are not significant. However, when exact decimal representation is needed, such as in financial calculations, you can use the decimal module.


In [45]:
from decimal import Decimal
print(Decimal(".1") + Decimal(".2"))

0.3


## Roundoff Error

100,000,000: 0 10011001 (153) 01111101011110000100000 
10: 0 10000010(130) 01000000000000000000000
Then we need to do Exponent Alignment: 23 = 153 - 130, which means 1.01000000000000000000000 should shift right 23 bits. 
then we have 01111101011110000100000 0000 + 00000000000000000000000 1010 =  01111101011110000100000 1010

According to Round to Nearest, Ties to Even: finally we have 01111101011110000100001

In [46]:
print(format(np.float32(100000000) + np.float32(10),'10.3f'))

100000008.000


**Kahan summation algorithm `math.fsum()` can effectively solve this roundoff errors.**

In [48]:
import math
math.fsum([np.float32(100000000), np.float32(10)])

100000010.0

## Cancellation Error
- When two nearly equal floating point numbers are subtracted. 

0.1111112 = 0 01111011(123) 11000111000111001000101
0.1111111 = 0 01111011(123) 11000111000111000110111
```
  Significand of 0.1111112: 1.11000111000111001000101
- Significand of 0.1111111: 1.11000111000111000110111
-------------------------------------------------
  Resulting Significand:    0.00000000000000000001110
``` 
Then we need to left shift 20-bit to get 1.11

the original exp = -4 then we need to let exp - 20 = -24 as the result exponent

-24 = 103 - 127, so the exponent bits are 01100111

Finally, we get 0 0110011 111000000000000000000000 $\approx$ 1.04308128357e-07


In [50]:
print(format(np.float32(0.1111112) - np.float32(0.1111111), "1.17f"))

0.00000010430812836


# Solution to Errors
- When using floating point, we must check for approximate equality using `allclose` instead of `==`

`rtol`: relative tolerance 
`atol`: absolute tolerance

$$|a - b| \le (\text{atol} + \text{rtol} * |b|)$$ is True, then `np.allclose` returns True

In [53]:
import numpy as np 
a = np.float32(0.15 + 0.15)
b = np.float32(0.1 + 0.2)

if np.allclose(a, b, rtol = 1e-05, atol = 1e-08):
    print("a == b")
else:
    print("a != b")

a == b
